In [1]:
!pip install tqdm


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\visha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension install --py widgetsnbextension --symlink --sys-prefix


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Compu

In [3]:
# import neccessary packages
import random
import math
import networkx as nx
import pandas as pd
import numpy as np
# from tqdm.notebook import tqdm_notebook
from collections import defaultdict
import time
from scipy.spatial.distance import cdist
import plotly.express as px
import pygad
from tqdm.notebook import tqdm_notebook

In [4]:
tilbury_df = pd.read_excel('C:/Users/visha/Downloads/aco/tilbury_data_new.xlsx', sheet_name='final')
tilbury_df['n1'] = tilbury_df['n1'].astype(str)
tilbury_df['n2'] = tilbury_df['n2'].astype(str)
tilbury_df['edge_key'] = tilbury_df['n1']+"-"+tilbury_df['n2']
tilbury_df['weight'] = tilbury_df['weight'].astype(float)
tilbury_df['edge_stat'] = tilbury_df['edge_stat'].astype(str)

tt = pd.DataFrame(tilbury_df[['n1','n2','edge_stat','edge_key','weight']])

target_segments = tt[tt["edge_stat"] == "target"]["edge_key"].tolist()
normal_segments = tt[tt["edge_stat"] == "normal"]["edge_key"].tolist()

modified = []
for i in target_segments:
    for j in i.split("-"):
        modified.append(j)

modified = set(modified.copy())
modified = list(modified)

cols = ['n1', 'n2', 'weight']
df_graph = tt[cols]
g = nx.DiGraph()
g.add_weighted_edges_from(df_graph.values)

In [68]:
# Creating distance matrix from target to target nodes

import networkx as nx
import pandas as pd
from tqdm.notebook import tqdm_notebook

# Initialize data structures for results
dynamic_targets = modified.copy()
distances_df = pd.DataFrame(index=dynamic_targets, columns=dynamic_targets, dtype=float)
path_details = []  # To store from_node, to_node, distance, path


def calculate_all_paths():
    total_pairs = len(dynamic_targets) * (len(dynamic_targets) - 1)  # Total pairs excluding self-loops
    progress_bar = tqdm_notebook(total=total_pairs, desc="Calculating paths", unit="pair")

    for s in dynamic_targets:
        for t in dynamic_targets:
            if s != t:  # No need to compute paths to itself
                try:
                    # Compute shortest path
                    path = nx.shortest_path(g, source=s, target=t, weight="weight", method="dijkstra")
                    
                    # Calculate path length (sum of weights along the path)
                    path_length = sum(
                        g[u][v]["weight"] for u, v in zip(path[:-1], path[1:])
                    )

                    # Update distances_df
                    distances_df.loc[s, t] = path_length

                    # Append details for path_details_df
                    path_details.append({
                        "from_node": s,
                        "to_node": t,
                        "distance": path_length,
                        "path": path
                    })
                except nx.NetworkXNoPath:
                    # If no path, leave distance_df as NaN (default)
                    continue
                finally:
                    # Update the progress bar
                    progress_bar.update(1)

    progress_bar.close()


def save_results():
    # Convert path details into a DataFrame
    path_details_df = pd.DataFrame(path_details)

    print("Distances DataFrame:")
    print(distances_df)
    print("\nPath Details DataFrame:")
    print(path_details_df)

    return distances_df, path_details_df


# Example usage:
calculate_all_paths()
distances_df, path_details_df = save_results()


Calculating paths:   0%|          | 0/511940 [00:00<?, ?pair/s]

Distances DataFrame:
                      53893715756187830  6905705434121322047  \
53893715756187830                   NaN                  NaN   
6905705434121322047         3067.902855                  NaN   
7612750293809454279         3834.770664          1203.283733   
12623676994084260887        3779.665501          2283.501963   
7076351127492486821         3541.472435          1237.727006   
...                                 ...                  ...   
2624383411529082061         3626.279191          1036.378747   
5866583667213165172         3421.924104          1454.621883   
13890494892616266481        3725.690856          1094.203925   
5183385557346899990         4076.921749          2580.758212   
1410181835835205265         2680.809260           796.927319   

                      7612750293809454279  12623676994084260887  \
53893715756187830                     NaN                   NaN   
6905705434121322047           1239.889336           2055.365437   
761275029

In [6]:
print(path_details_df.head())
print((path_details_df.shape))

print(distances_df.head())
print((distances_df.shape))


             from_node               to_node     distance  \
0    53893715756187830   6171979078522256589    50.545461   
1    53893715756187830  14687997483004889694    88.800569   
2    53893715756187830   6503800644946969504   141.510562   
3  6905705434121322047     53893715756187830  3067.902855   
4  6905705434121322047   7612750293809454279  1239.889336   

                                                path  
0           [53893715756187830, 6171979078522256589]  
1  [53893715756187830, 6171979078522256589, 14687...  
2  [53893715756187830, 6171979078522256589, 14687...  
3  [6905705434121322047, 13624383220648599578, 98...  
4  [6905705434121322047, 13624383220648599578, 51...  
(502019, 4)
                      53893715756187830  6905705434121322047  \
53893715756187830                   NaN                  NaN   
6905705434121322047         3067.902855                  NaN   
7612750293809454279         3834.770664          1203.283733   
12623676994084260887        3779.66

In [7]:
# target_distances = distances_df.set_index(list(distances_df)[0])
# print(target_distances.head())
target_distances = distances_df.copy()
# Convert row labels (index) to strings
target_distances.index = target_distances.index.astype(str)

print("Unique Row Labels:", target_distances.index.nunique())
print("Unique Column Labels:", target_distances.columns.nunique())

# Check for mismatches
assert set(target_distances.index) == set(target_distances.columns), "Row and column labels do not match!"

print("Row Labels Type:", type(target_distances.index[0]))
print("Column Labels Type:", type(target_distances.columns[0]))

Unique Row Labels: 716
Unique Column Labels: 716
Row Labels Type: <class 'str'>
Column Labels Type: <class 'str'>


In [8]:
G = nx.DiGraph()

# Add edges with conditions
for source in target_distances.index:
    for target in target_distances.columns:
        distance = target_distances.loc[source, target]
        if pd.notna(distance) and distance != 0 and np.isfinite(distance):
            G.add_edge(source, target, weight=distance)

# Print counts
print("Number of Nodes:", G.number_of_nodes())
print("Number of Edges:", G.number_of_edges())

Number of Nodes: 716
Number of Edges: 502019


In [59]:
def initialize_pheromone(graph, init_pheromone, node_mapping, target_df, extra_pheromone):
    """
    Initialize pheromone level for every edge in the graph.
    Optimized using NumPy arrays.
    
    Parameters:
    - graph: The graph structure (not used directly in this function but kept for context).
    - init_pheromone: The initial pheromone level for all edges.
    - node_mapping: A dictionary mapping node identifiers to indices in the pheromone matrix.
    - target_df: A DataFrame containing columns 'n1' and 'n2' representing target edges.
    - extra_pheromone: The additional pheromone to assign to target edges.
    
    Returns:
    - pheromone: A 2D NumPy array representing the pheromone levels for each edge.
    """
    num_nodes = len(node_mapping)
    pheromone = np.full((num_nodes, num_nodes), init_pheromone, dtype=np.float32)
    
    # Add extra pheromone for edges in target_df
    for _, row in target_df.iterrows():
        n1 = row['n1']
        n2 = row['n2']
        if n1 in node_mapping and n2 in node_mapping:
            i, j = node_mapping[n1], node_mapping[n2]
            pheromone[i, j] += extra_pheromone
    
    return pheromone

def create_ants(n_ants, graph, min_visit_percentage=0.6, node_mapping=None):
    """
    Create ants with tracking of minimum required visits.
    Optimized with NumPy-friendly structures.
    """
    nodes_list = np.array(list(node_mapping.keys()))  # Use original nodes
    num_nodes = len(nodes_list)
    min_required_visits = int(num_nodes * min_visit_percentage)

    ants = {
        'path': [[] for _ in range(n_ants)],
        'visited_nodes': [set() for _ in range(n_ants)],
        'path_edges': [[] for _ in range(n_ants)],
        'stack': [[] for _ in range(n_ants)],
        'visit_count': np.zeros((n_ants, num_nodes), dtype=int),
        'min_required_visits': min_required_visits
    }

    for i in range(n_ants):
        start_node = random.choice(nodes_list)
        ants['path'][i].append(start_node)
        ants['visited_nodes'][i].add(start_node)
        ants['stack'][i].append(start_node)
        ants['visit_count'][i, node_mapping[start_node]] = 1  # Use mapped index

    return ants

def select_next_node(ant_idx, ants, graph, pheromone, heuristic, alpha, unvisited_only=True, max_visits=10, node_mapping=None):
    """
    Select the next node using cached heuristic values and ACO probability rules.
    """
    current_node = ants['path'][ant_idx][-1]
    current_node_idx = node_mapping[current_node]

    neighbors = list(graph.successors(current_node))
    if unvisited_only:
        neighbors = [n for n in neighbors if n not in ants['visited_nodes'][ant_idx]]
    else:
        neighbors = [n for n in neighbors if ants['visit_count'][ant_idx, node_mapping[n]] < max_visits]

    if not neighbors:
        return None

    neighbor_indices = [node_mapping[n] for n in neighbors]

    # Use cached heuristic values
    tau = pheromone[current_node_idx, neighbor_indices] ** alpha
    eta = heuristic[current_node_idx, neighbor_indices]
    weights = tau * eta

    total = weights.sum()
    if total <= 1e-12:
        return None

    # Roulette wheel selection
    probabilities = weights / total
    return np.random.choice(neighbors, p=probabilities)

def precompute_heuristic(graph, node_mapping, beta=2.0):
    """
    Precompute heuristic values (1 / weight ** beta) for all edges in the graph.
    Returns a 2D NumPy array indexed by node indices.
    """
    num_nodes = len(node_mapping)
    heuristic = np.zeros((num_nodes, num_nodes), dtype=np.float32)

    for u, v, data in graph.edges(data=True):
        u_idx = node_mapping[u]
        v_idx = node_mapping[v]
        weight = data.get('weight', 1.0)
        if weight > 0:
            heuristic[u_idx, v_idx] = (1.0 / weight) ** beta
        else:
            heuristic[u_idx, v_idx] = 0.0  # Handle zero or invalid weights safely

    return heuristic


def move_ant(ant_idx, ants, next_node, node_mapping):
    """
    Optimized move operation for ants using node_mapping.
    """
    current_node = ants['path'][ant_idx][-1]
    ants['path'][ant_idx].append(next_node)
    ants['visited_nodes'][ant_idx].add(next_node)
    ants['path_edges'][ant_idx].append((current_node, next_node))
    ants['stack'][ant_idx].append(next_node)

    # Update visit count using mapped indices
    next_node_idx = node_mapping[next_node]
    ants['visit_count'][ant_idx, next_node_idx] += 1

def evaporate_pheromone(pheromone, evaporation_rate):
    """
    Optimized pheromone evaporation using NumPy.
    """
    pheromone *= (1.0 - evaporation_rate)

def compute_path_cost(path_edges, graph, node_mapping):
    """
    Compute the total cost of a path using the graph weights.
    Uses node_mapping to translate nodes to their indices.
    """
    total_cost = 0.0
    for u, v in path_edges:
        total_cost += graph[u][v].get('weight', 1.0)
    return total_cost


In [61]:
def process_top_ants(
    ants, graph, pheromone, node_mapping, compute_path_cost,
    top_percent=0.3, deposit_factor=1.0, virtual_pheromone_weight=0.1
):
    """
    Process the top-performing ants:
    1. Deposit pheromones based on their paths.
    2. Reweight pheromones for unvisited nodes.
    3. Return top ants' paths and costs.
    """
    # Compute scores for ants
    scores = [
        (len(set(ants['path'][i])), -compute_path_cost(ants['path_edges'][i], graph, node_mapping))
        for i in range(len(ants['path']))
    ]

    # Sort indices of ants based on their scores
    sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

    # Select the top-performing ants
    n_top = max(1, int(len(sorted_indices) * top_percent))
    top_indices = sorted_indices[:n_top]

    top_paths = []
    top_costs = []

    # Deposit pheromone for the top ants
    visited_by_top_ants = set()
    for idx in top_indices:
        path = ants['path'][idx]
        path_edges = ants['path_edges'][idx]
        cost = compute_path_cost(path_edges, graph, node_mapping)

        top_paths.append(path)
        top_costs.append(cost)

        path_len = len(path) - 1
        if path_len < 1:
            continue
        contribution = deposit_factor / path_len
        for u, v in path_edges:
            # Map nodes to indices
            u_idx = node_mapping[u]
            v_idx = node_mapping[v]
            pheromone[u_idx, v_idx] += contribution
        visited_by_top_ants.update(ants['visited_nodes'][idx])

    # Identify unvisited nodes
    all_nodes = set(graph.nodes)
    unvisited_nodes = all_nodes - visited_by_top_ants

    # Reweight pheromones for unvisited nodes
    for node in unvisited_nodes:
        idx = node_mapping[node]
        pheromone[idx, :] += virtual_pheromone_weight
        pheromone[:, idx] += virtual_pheromone_weight

    return top_paths, top_costs

def run_aco(
    graph,
    target_df,
    extra_pheromone = 0.01,
    n_ants=10,
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.5,
    init_pheromone=0.01,
    iterations=10,
    unvisited_only=True,
    max_visits=10,
    min_visit_percentage=0.6,
    top_percent=0.3,
    virtual_pheromone_weight=0.1
):
    """
    Main ACO loop with heuristic caching for improved performance.
    """
    node_mapping = {node: idx for idx, node in enumerate(graph.nodes)}
    reverse_mapping = {idx: node for node, idx in node_mapping.items()}
    pheromone = initialize_pheromone(graph, init_pheromone, node_mapping,target_df, extra_pheromone)

    # Precompute heuristic values
    heuristic = precompute_heuristic(graph, node_mapping, beta)

    global_best_path = None
    global_best_cost = float('inf')
    iteration_best_paths = []
    iteration_best_costs = []
    top_ants_data = []

    total_start_time = time.time()

    for it in range(iterations):
        ants = create_ants(n_ants, graph, min_visit_percentage=min_visit_percentage, node_mapping=node_mapping)
        unvisited_only = True  # it < (0.75 * iterations)
        for ant_idx in range(n_ants):
            while True:  # len(ants['visited_nodes'][ant_idx]) < ants['min_required_visits']:
                next_node = select_next_node(
                    ant_idx, ants, graph, pheromone, heuristic, alpha,
                    unvisited_only=unvisited_only, max_visits=max_visits, node_mapping=node_mapping
                )
                if next_node is not None:
                    move_ant(ant_idx, ants, next_node, node_mapping)  # Pass node_mapping here
                else:
                    break

        # Get top ants' paths and costs
        top_paths, top_costs = process_top_ants(
            ants, graph, pheromone, node_mapping, compute_path_cost,
            top_percent=top_percent, deposit_factor=1.0, virtual_pheromone_weight=virtual_pheromone_weight
        )

        evaporate_pheromone(pheromone, evaporation_rate)

        # Identify the iteration best
        iteration_best_ant = max(
            range(len(ants['path'])),
            key=lambda idx: (len(set(ants['path'][idx])), -compute_path_cost(ants['path_edges'][idx], graph, node_mapping))
        )
        iteration_best_path = ants['path'][iteration_best_ant]
        iteration_best_cost = compute_path_cost(ants['path_edges'][iteration_best_ant], graph, node_mapping)

        # Translate nodes in iteration_best_path to original node labels using node_mapping and reverse_mapping
        iteration_best_paths.append([reverse_mapping[node_mapping[node]] for node in iteration_best_path])
        iteration_best_costs.append(iteration_best_cost)

        # Update global best if the iteration best is better
        distinct_nodes_iteration = len(set(iteration_best_path))
        distinct_nodes_global = len(set(global_best_path or []))

        if distinct_nodes_iteration > distinct_nodes_global:
            global_best_path = iteration_best_path
            global_best_cost = iteration_best_cost
        elif distinct_nodes_iteration == distinct_nodes_global:
            if iteration_best_cost < global_best_cost:
                global_best_path = iteration_best_path
                global_best_cost = iteration_best_cost

        # Store top ants' data for this iteration
        top_ants_data.append({
            "iteration": it + 1,
            "top_n_paths": [[reverse_mapping[node_mapping[node]] for node in path] for path in top_paths],
            "top_n_cost": top_costs
        })

        print(f"Iteration {it+1}/{iterations} - Iteration Best Path: {len(set(iteration_best_path))} nodes, "
              f"Cost: {iteration_best_cost:.4f}, Global Best Cost: {global_best_cost:.4f}")

    total_time = time.time() - total_start_time
    print(f"Total time: {total_time:.4f}s")

    # Create the DataFrame from top_ants_data
    top_ants_df = pd.DataFrame(top_ants_data)

    return {
        "global_best_path": [reverse_mapping[node_mapping[node]] for node in global_best_path],
        "global_best_cost": global_best_cost,
        "iteration_best_paths": iteration_best_paths,
        "iteration_best_costs": iteration_best_costs,
        "top_ants_df": top_ants_df
    }



In [62]:
# Main ACO iteration path - lomgest possible path

result = run_aco(
    graph=G,
    target_df= tt[tt["edge_stat"] == "target"],
    extra_pheromone= 0.5,
    n_ants=1000,
    alpha=1.0,
    beta=3.0,
    evaporation_rate=0.5,
    init_pheromone=0.5,
    iterations=35,
    max_visits=50,
    min_visit_percentage =1.5,
    top_percent=0.25,
    virtual_pheromone_weight=0.4
)


# print("Global Best Path:", result["global_best_path"])
print("Global Best Cost:", result["global_best_cost"])
# print("Iteration Best Paths:", result["iteration_best_paths"])
print("Iteration Best Costs:", result["iteration_best_costs"])


Iteration 1/35 - Iteration Best Path: 561 nodes, Cost: 74950.1001, Global Best Cost: 74950.1001
Iteration 2/35 - Iteration Best Path: 569 nodes, Cost: 76565.4449, Global Best Cost: 76565.4449
Iteration 3/35 - Iteration Best Path: 595 nodes, Cost: 75131.6601, Global Best Cost: 75131.6601
Iteration 4/35 - Iteration Best Path: 608 nodes, Cost: 70226.8017, Global Best Cost: 70226.8017
Iteration 5/35 - Iteration Best Path: 574 nodes, Cost: 71307.7351, Global Best Cost: 70226.8017
Iteration 6/35 - Iteration Best Path: 585 nodes, Cost: 58776.2668, Global Best Cost: 70226.8017
Iteration 7/35 - Iteration Best Path: 589 nodes, Cost: 53699.7011, Global Best Cost: 70226.8017
Iteration 8/35 - Iteration Best Path: 576 nodes, Cost: 59939.4769, Global Best Cost: 70226.8017
Iteration 9/35 - Iteration Best Path: 629 nodes, Cost: 56811.1243, Global Best Cost: 56811.1243
Iteration 10/35 - Iteration Best Path: 620 nodes, Cost: 56356.6130, Global Best Cost: 56811.1243
Iteration 11/35 - Iteration Best Path: 

In [63]:
# Main ACO iteration path - lomgest possible path

result2 = run_aco(
    graph=G,
    target_df= tt[tt["edge_stat"] == "target"],
    extra_pheromone= 0.35,
    n_ants=2000,
    alpha=1.0,
    beta=3.0,
    evaporation_rate=0.5,
    init_pheromone=0.5,
    iterations=35,
    max_visits=50,
    min_visit_percentage =1.5,
    top_percent=0.25,
    virtual_pheromone_weight=0.4
)


# print("Global Best Path:", result["global_best_path"])
print("Global Best Cost:", result2["global_best_cost"])
# print("Iteration Best Paths:", result["iteration_best_paths"])
print("Iteration Best Costs:", result2["iteration_best_costs"])


Iteration 1/35 - Iteration Best Path: 589 nodes, Cost: 77617.8491, Global Best Cost: 77617.8491
Iteration 2/35 - Iteration Best Path: 598 nodes, Cost: 73053.9772, Global Best Cost: 73053.9772
Iteration 3/35 - Iteration Best Path: 620 nodes, Cost: 82596.4442, Global Best Cost: 82596.4442
Iteration 4/35 - Iteration Best Path: 601 nodes, Cost: 65785.1371, Global Best Cost: 82596.4442
Iteration 5/35 - Iteration Best Path: 582 nodes, Cost: 57564.9884, Global Best Cost: 82596.4442
Iteration 6/35 - Iteration Best Path: 601 nodes, Cost: 65117.3354, Global Best Cost: 82596.4442
Iteration 7/35 - Iteration Best Path: 592 nodes, Cost: 60128.2518, Global Best Cost: 82596.4442
Iteration 8/35 - Iteration Best Path: 622 nodes, Cost: 59288.9790, Global Best Cost: 59288.9790
Iteration 9/35 - Iteration Best Path: 604 nodes, Cost: 53856.0545, Global Best Cost: 59288.9790
Iteration 10/35 - Iteration Best Path: 597 nodes, Cost: 51288.3481, Global Best Cost: 59288.9790
Iteration 11/35 - Iteration Best Path: 

In [64]:
# Main ACO iteration path - lomgest possible path

result3 = run_aco(
    graph=G,
    target_df= tt[tt["edge_stat"] == "target"],
    extra_pheromone= 0.65,
    n_ants=2000,
    alpha=1.0,
    beta=3.0,
    evaporation_rate=0.5,
    init_pheromone=0.5,
    iterations=35,
    max_visits=50,
    min_visit_percentage =1.5,
    top_percent=0.25,
    virtual_pheromone_weight=0.4
)


# print("Global Best Path:", result["global_best_path"])
print("Global Best Cost:", result3["global_best_cost"])
# print("Iteration Best Paths:", result["iteration_best_paths"])
print("Iteration Best Costs:", result3["iteration_best_costs"])


Iteration 1/35 - Iteration Best Path: 626 nodes, Cost: 75275.6201, Global Best Cost: 75275.6201
Iteration 2/35 - Iteration Best Path: 568 nodes, Cost: 72909.3931, Global Best Cost: 75275.6201
Iteration 3/35 - Iteration Best Path: 605 nodes, Cost: 72907.0140, Global Best Cost: 75275.6201
Iteration 4/35 - Iteration Best Path: 601 nodes, Cost: 69745.8951, Global Best Cost: 75275.6201
Iteration 5/35 - Iteration Best Path: 577 nodes, Cost: 66385.4066, Global Best Cost: 75275.6201
Iteration 6/35 - Iteration Best Path: 600 nodes, Cost: 61051.3972, Global Best Cost: 75275.6201
Iteration 7/35 - Iteration Best Path: 606 nodes, Cost: 56329.2038, Global Best Cost: 75275.6201
Iteration 8/35 - Iteration Best Path: 605 nodes, Cost: 58118.5408, Global Best Cost: 75275.6201
Iteration 9/35 - Iteration Best Path: 623 nodes, Cost: 53066.5803, Global Best Cost: 75275.6201
Iteration 10/35 - Iteration Best Path: 595 nodes, Cost: 53067.6514, Global Best Cost: 75275.6201
Iteration 11/35 - Iteration Best Path: 

In [25]:
# Second ACO
def run_aco_rerun(
    graph,
    best_path,
    n_ants=10,
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.5,
    init_pheromone=0.01,
    iterations=10,
    unvisited_only=True,
    max_visits=10,
    top_percent=0.3,
    virtual_pheromone_weight=0.1
):
    """
    Re-run ACO with the nodes in the best_path excluded from the graph.
    """
    # Exclude nodes in best_path from the graph
    remaining_nodes = set(graph.nodes) - set(best_path)
    subgraph = graph.subgraph(remaining_nodes).copy()

    # Re-map nodes for the subgraph
    node_mapping = {node: idx for idx, node in enumerate(subgraph.nodes)}
    reverse_mapping = {idx: node for node, idx in node_mapping.items()}
    pheromone = initialize_pheromone(subgraph, init_pheromone, node_mapping)

    # Precompute heuristic values for the subgraph
    heuristic = precompute_heuristic(subgraph, node_mapping, beta)

    global_best_path = None
    global_best_cost = float('inf')
    iteration_best_paths = []
    iteration_best_costs = []

    total_start_time = time.time()

    for it in range(iterations):
        ants = create_ants(n_ants, subgraph, node_mapping=node_mapping)
        unvisited_only = True  # Keep True or dynamic as needed

        for ant_idx in range(n_ants):
            while True:
                next_node = select_next_node(
                    ant_idx, ants, subgraph, pheromone, heuristic, alpha,
                    unvisited_only=unvisited_only, max_visits=max_visits, node_mapping=node_mapping
                )
                if next_node is not None:
                    move_ant(ant_idx, ants, next_node, node_mapping)
                else:
                    break

        process_top_ants(
            ants, subgraph, pheromone, node_mapping, compute_path_cost,
            top_percent=top_percent, deposit_factor=1.0, virtual_pheromone_weight=virtual_pheromone_weight
        )

        evaporate_pheromone(pheromone, evaporation_rate)

        # Identify the iteration best
        iteration_best_ant = max(
            range(len(ants['path'])),
            key=lambda idx: (len(set(ants['path'][idx])), -compute_path_cost(ants['path_edges'][idx], subgraph, node_mapping))
        )
        iteration_best_path = ants['path'][iteration_best_ant]
        iteration_best_cost = compute_path_cost(ants['path_edges'][iteration_best_ant], subgraph, node_mapping)

        # Translate nodes in iteration_best_path to original node labels
        iteration_best_paths.append([reverse_mapping[node_mapping[node]] for node in iteration_best_path])
        iteration_best_costs.append(iteration_best_cost)

        # Update global best if the iteration best is better
        distinct_nodes_iteration = len(set(iteration_best_path))
        distinct_nodes_global = len(set(global_best_path or []))

        if distinct_nodes_iteration > distinct_nodes_global:
            global_best_path = iteration_best_path
            global_best_cost = iteration_best_cost
        elif distinct_nodes_iteration == distinct_nodes_global:
            if iteration_best_cost < global_best_cost:
                global_best_path = iteration_best_path
                global_best_cost = iteration_best_cost

        print(f"Iteration {it+1}/{iterations} - Iteration Best Path: {len(set(iteration_best_path))} nodes, "
              f"Cost: {iteration_best_cost:.4f}, Global Best Cost: {global_best_cost:.4f}")

    total_time = time.time() - total_start_time
    print(f"Total time: {total_time:.4f}s")

    return {
        "global_best_path": [reverse_mapping[node_mapping[node]] for node in global_best_path],
        "global_best_cost": global_best_cost,
        "iteration_best_paths": iteration_best_paths,
        "iteration_best_costs": iteration_best_costs,
    }


In [26]:
best_path = result["global_best_path"]

# subrtact 1st ACO ressult from new second graph
remaining_result = run_aco_rerun(graph=G, 
                                best_path=best_path,
                                n_ants=7000,
                                alpha=1.0,
                                beta=2.8,
                                evaporation_rate=0.6,
                                init_pheromone=0.5,
                                iterations=100,
                                max_visits=60,
                                top_percent=0.3,
                                virtual_pheromone_weight=0.5)

print("Remaining Nodes Global Best Path:", remaining_result["global_best_path"])
print("Remaining Nodes Global Best Cost:", remaining_result["global_best_cost"])
len(remaining_result["global_best_path"])

Iteration 1/100 - Iteration Best Path: 76 nodes, Cost: 26634.8379, Global Best Cost: 26634.8379
Iteration 2/100 - Iteration Best Path: 60 nodes, Cost: 15159.1157, Global Best Cost: 26634.8379
Iteration 3/100 - Iteration Best Path: 53 nodes, Cost: 13950.5075, Global Best Cost: 26634.8379
Iteration 4/100 - Iteration Best Path: 59 nodes, Cost: 14500.7305, Global Best Cost: 26634.8379
Iteration 5/100 - Iteration Best Path: 55 nodes, Cost: 14536.7319, Global Best Cost: 26634.8379
Iteration 6/100 - Iteration Best Path: 52 nodes, Cost: 12139.7883, Global Best Cost: 26634.8379
Iteration 7/100 - Iteration Best Path: 55 nodes, Cost: 15310.4206, Global Best Cost: 26634.8379
Iteration 8/100 - Iteration Best Path: 55 nodes, Cost: 13968.1350, Global Best Cost: 26634.8379
Iteration 9/100 - Iteration Best Path: 52 nodes, Cost: 13429.2107, Global Best Cost: 26634.8379
Iteration 10/100 - Iteration Best Path: 47 nodes, Cost: 8951.3526, Global Best Cost: 26634.8379
Iteration 11/100 - Iteration Best Path: 

78

In [44]:
def construct_full_path(final_path, path_details_df):
    """
    Constructs the full path list based on the final path list and path details DataFrame.

    Parameters:
    - final_path (list): The list of nodes representing the main path sequence (e.g., ["A", "D", "F"]).
    - path_details_df (pd.DataFrame): DataFrame containing `from_node`, `to_node`, and `path`.
      Each row represents a subpath with nodes connecting `from_node` to `to_node`.

    Returns:
    - list: Full path list combining all subpaths from path_details_df.

    Raises:
    - ValueError: If a subpath for a pair of nodes is not found in path_details_df.
    """
    full_path = []

    # Loop through consecutive nodes in the final_path
    for i in range(len(final_path) - 1):
        current_node = final_path[i]
        next_node = final_path[i + 1]
        
        # Find the subpath in the DataFrame
        subpath_row = path_details_df[
            (path_details_df["from_node"] == current_node) &
            (path_details_df["to_node"] == next_node)
        ]
        
        if not subpath_row.empty:
            subpath = subpath_row.iloc[0]["path"]
            # Add the subpath to the full path, avoiding duplicate nodes
            if full_path:
                full_path.extend(subpath[1:])  # Skip the first node of the subpath
            else:
                full_path.extend(subpath)  # Add the entire first subpath
        else:
            raise ValueError(f"No subpath found for {current_node} -> {next_node}")

    return full_path

In [ ]:
full_path_1 = construct_full_path(final_path=result['global_best_path'], path_details_df=path_details_df)

In [51]:
def create_ordered_dataframe(full_path, original_df):
    """
    Creates a new DataFrame with rows from `original_df` ordered according to the sequence in `full_path`.

    Parameters:
    - full_path (list): List of nodes defining the order (e.g., ["A", "D", "F"]).
    - original_df (pd.DataFrame): Original DataFrame with columns `from_node`, `to_node`, `weight`, and `source`.

    Returns:
    - pd.DataFrame: New DataFrame with rows ordered according to `full_path`.
    """
    # Create an empty list to store matching rows
    ordered_rows = []

    # Iterate through consecutive pairs in full_path
    for i in range(len(full_path) - 1):
        current_node = full_path[i]
        next_node = full_path[i + 1]
        
        # Find the corresponding row in the original DataFrame
        row = original_df[
            (original_df["n1"] == current_node) &
            (original_df["n2"] == next_node)
        ]
        
        if not row.empty:
            ordered_rows.append(row.iloc[0])  # Append the first matching row
        else:
            raise ValueError(f"No row found for {current_node} -> {next_node}")

    # Create a new DataFrame from the ordered rows
    new_df = pd.DataFrame(ordered_rows)
    return new_df

In [ ]:
full_path_df = create_ordered_dataframe(full_path=full_path_1, original_df=tilbury_df)

In [57]:
target_count = full_path_df[full_path_df['edge_stat'] == "target"].drop_duplicates().shape[0]
target_count

480

In [65]:
full_path_01 = construct_full_path(final_path=result['global_best_path'], path_details_df=path_details_df)
full_path_df1 = create_ordered_dataframe(full_path=full_path_01, original_df=tilbury_df)
target_count1 = full_path_df1[full_path_df1['edge_stat'] == "target"].drop_duplicates().shape[0]
target_count1

463

In [66]:
full_path_2 = construct_full_path(final_path=result2['global_best_path'], path_details_df=path_details_df)
full_path_df2 = create_ordered_dataframe(full_path=full_path_2, original_df=tilbury_df)
target_count2 = full_path_df2[full_path_df2['edge_stat'] == "target"].drop_duplicates().shape[0]
target_count2

459

In [67]:
full_path_3 = construct_full_path(final_path=result3['global_best_path'], path_details_df=path_details_df)
full_path_df3 = create_ordered_dataframe(full_path=full_path_3, original_df=tilbury_df)
target_count3 = full_path_df3[full_path_df3['edge_stat'] == "target"].drop_duplicates().shape[0]
target_count3

438

In [48]:
len(full_path_1)

1596

In [27]:
all_nodes = set(G.nodes)
nodes_in_best_path = set(best_path)
nodes_in_second_path = set(remaining_result["global_best_path"])

visited_nodes_1 = len(nodes_in_best_path)
visited_nodes_2 = len(nodes_in_second_path)
left_out_nodes_1 = len(all_nodes-nodes_in_best_path)
left_out_nodes_2 = len(all_nodes-nodes_in_best_path-nodes_in_second_path)
print(visited_nodes_1, left_out_nodes_1, visited_nodes_2, left_out_nodes_2)
print(f"total covered: {visited_nodes_1+visited_nodes_2}, {(visited_nodes_1+visited_nodes_2)/len(all_nodes)}")

613 103 78 25
total covered: 691, 0.9650837988826816


In [ ]:
# dynamic_targets = modified.copy()
# target_sequence = []
# final_paths = []
# progress = []

# def pathGenerator(s):
#     final_paths.append(s)
#     current_node = s
#     bar_format = '{percentage:6.2f}%{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'

#     while dynamic_targets:
#         #for i in tqdm_notebook(range(len(modified)), desc = 'Path Generation', bar_format = bar_format):
#             try:
#                 next_node = allPossibleTarget(current_node)
#                 if next_node is None:
#                     print(f"No path found from {current_node}")
#                     return final_paths
                
#                 path = targetPath(current_node, next_node)
#                 final_paths.append(path)
#                 target_sequence.append(next_node)
#                 dynamic_targets.remove(next_node)
#                 current_node = next_node
#                 progress.append(i)
#             except:
#                 print("check J")
#                 return final_paths
#     return final_paths


# def allPossibleTarget(s):
#     shortest_paths = {}
#     for t in dynamic_targets:
#         try:
#             path_length = nx.shortest_path_length(g, source = s, target = t, weight = "weight", method = 'dijkstra')
#             shortest_paths[t] = path_length
#             print(shortest_paths[t])
#         except nx.NetworkXNoPath:
#             print(f"No path found from {s} to {t}")

#     if not shortest_paths:
#         return None
    
#     next_shortest_node = pd.DataFrame(shortest_paths.items(), columns=['nodes', 'distance']).sort_values(by = ' distance', ascending=True).reset_index().iloc[1,1]
#     return next_shortest_node

# def targetPath(s,t):
#     path = nx.shortest_path(g, source = s, target = t, weight = "weight", method = 'dijkstra')
#     return path
            


In [35]:
# import networkx as nx

# def find_minimum_path_and_cost(g, source, target):
#     """
#     Return the minimum-weight path and its total cost from 'source' to 'target' 
#     using Dijkstra's algorithm. If no path exists, return (None, None).
#     """
#     try:
#         # Find the shortest path (by edge weight)
#         path = nx.shortest_path(
#             G=g, 
#             source=source, 
#             target=target, 
#             weight='weight', 
#             method='dijkstra'
#         )
        
#         # Calculate total cost by summing edge weights along the path
#         total_cost = 0
#         for i in range(len(path) - 1):
#             u = path[i]
#             v = path[i+1]
#             total_cost += g[u][v]['weight']
        
#         return path, total_cost

#     except nx.NetworkXNoPath:
#         print(f"No path found from {source} to {target}")
#         return None, None


In [43]:
# from math import inf

# # Replace None with infinity, so they never become the minimum
# adjusted_costs = [c if c is not None else inf for c in costs]

# min_val = min(adjusted_costs)
# index_val = adjusted_costs.index(min_val)
# print(f"Minimum cost = {min_val}, at index = {index_val}")


Minimum cost = 3253631480.4738326, at index = 13


In [41]:
# left_out_nodes = all_nodes-nodes_in_best_path
# paths =[]
# costs = []
# for x in left_out_nodes:
#     path_1, cost_1 = find_minimum_path_and_cost(G, source='15647424580509425104', target=x)
#     paths.append(path_1)
#     costs.append(cost_1)

No path found from 15647424580509425104 to 15006999009278211906
No path found from 15647424580509425104 to 13940021069375013494
No path found from 15647424580509425104 to 11560310318897117802
No path found from 15647424580509425104 to 16015658737375671042
No path found from 15647424580509425104 to 6967225432775730970
No path found from 15647424580509425104 to 8756251546489759879
No path found from 15647424580509425104 to 11570512215397646476
No path found from 15647424580509425104 to 7251382091613480086
No path found from 15647424580509425104 to 18111348986734965996


In [40]:
print(costs)

[np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.float64(3253635588.9825716), np.floa